In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/Daily_Public_Transport_Passenger_Journeys.csv')

In [ ]:
df

,Date,Local Route,Light Rail,Peak Service,Rapid Route,School,Other
0,30-08-2024,16436,10705,225,19026,3925,59.0
1,15-09-2023,15499,10671,267,18421,4519,61.0
2,28-12-2021,1756,2352,0,3775,0,13.0
3,11-01-2023,10536,8347,223,14072,0,48.0
4,11-09-2021,820,612,0,1283,0,11.0
...,...,...,...,...,...,...,...
1913,19-01-2023,11337,8671,262,14881,0,66.0
1914,16-08-2023,16791,11800,387,20936,4684,113.0
1915,10-01-2022,5732,3548,73,6644,0,25.0
1916,04-05-2020,3717,1952,70,4411,131,19.0


In [ ]:
null_counts = df.isnull().sum()
print("Count of null values in each column:\n", null_counts[null_counts > 0])

Count of null values in each column:
 Other    20
dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [ ]:
data = pd.read_csv('/content/Daily_Public_Transport_Passenger_Journeys.csv')
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True, errors='coerce')
print(data.head())

        Date  Local Route  Light Rail  Peak Service  Rapid Route  School  \
0 2024-08-30        16436       10705           225        19026    3925   
1 2023-09-15        15499       10671           267        18421    4519   
2 2021-12-28         1756        2352             0         3775       0   
3 2023-01-11        10536        8347           223        14072       0   
4 2021-09-11          820         612             0         1283       0   

   Other  
0   59.0  
1   61.0  
2   13.0  
3   48.0  
4   11.0  


In [ ]:
data['Other'] = data['Other'].fillna(data['Other'].median())


In [ ]:
print(data['Other'].isnull().sum())

0


In [ ]:
data['Total Passengers'] = data[['Local Route', 'Light Rail', 'Peak Service',
                                 'Rapid Route', 'School', 'Other']].sum(axis=1)

data['Month'] = data['Date'].dt.month
data['Day_of_Week'] = data['Date'].dt.dayofweek
data['Is_Weekend'] = (data['Day_of_Week'] >= 5).astype(int)

print(data[['Total Passengers', 'Month', 'Day_of_Week', 'Is_Weekend']].head())  # Verify new features


   Total Passengers  Month  Day_of_Week  Is_Weekend
0           50376.0      8            4           0
1           49438.0      9            4           0
2            7896.0     12            1           0
3           33226.0      1            2           0
4            2726.0      9            5           1


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# scaling the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
features = ['Month', 'Day_of_Week', 'Is_Weekend']
X = data[features]
routes = ['Local Route', 'Light Rail', 'Peak Service', 'Rapid Route', 'School', 'Other']
y = data[routes]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = Sequential([
    Dense(128, activation='relu', input_dim=X_train_scaled.shape[1], kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(len(routes))
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=64, validation_data=(X_test_scaled, y_test), callbacks=[early_stop])




Epoch 1/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 9710543.0000 - mae: 1906.1082 - val_loss: 8733092.0000 - val_mae: 1770.0483
Epoch 2/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 10503010.0000 - mae: 1972.9927 - val_loss: 8760997.0000 - val_mae: 1776.9481
Epoch 3/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 9284764.0000 - mae: 1856.7332 - val_loss: 8759840.0000 - val_mae: 1793.5889
Epoch 4/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 9308013.0000 - mae: 1861.6577 - val_loss: 8758181.0000 - val_mae: 1785.3883
Epoch 5/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 9972088.0000 - mae: 1918.6404 - val_loss: 8771805.0000 - val_mae: 1792.8849
Epoch 6/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 9403776.0000 - mae: 1869.0081 - val_loss: 8763146.0000 - val_mae: 1767.0292
Epoch 7/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 9637627.0000 - mae: 1893.8364 - val_loss: 8836333.0000 - val_mae: 1769.6841
Epoch 8/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - l

In [128]:
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8593622.0000 - mae: 1736.1327 
Test Loss: 8733093.0000, Test MAE: 1770.0482


In [127]:
def preprocess_input(user_input):
    input_data = pd.DataFrame([user_input], columns=['Month', 'Day_of_Week', 'Is_Weekend'])
    scaled_input = scaler.transform(input_data)
    return scaled_input

def predict_passenger_counts(user_input):
    processed_input = preprocess_input(user_input)
    predicted_values = model.predict(processed_input)
    predictions = dict(zip(routes, predicted_values[0]))
    return predictions

user_input = {
    'Month': 11,
    'Day_of_Week': 1,
    'Is_Weekend': 0
}

predicted_passengers = predict_passenger_counts(user_input)

for route, passengers in predicted_passengers.items():
    print(f"Predicted Total Passengers for {route}: {passengers:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Predicted Total Passengers for Local Route: 12892.04
Predicted Total Passengers for Light Rail: 8651.29
Predicted Total Passengers for Peak Service: 255.21
Predicted Total Passengers for Rapid Route: 15638.60
Predicted Total Passengers for School: 3322.24
Predicted Total Passengers for Other: 53.22
